In [ ]:
!pip install tts

In [ ]:
!tts

In [2]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.config.shared_configs import BaseAudioConfig
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.models.vits import Vits
from TTS.tts.configs.vits_config import VitsConfig
from trainer import Trainer, TrainerArgs
from TTS.tts.datasets import load_tts_samples

In [3]:
import os

In [4]:
!git clone https://github.com/hantswilliams/digitalclone-backend.git

Cloning into 'digitalclone-backend'...
remote: Enumerating objects: 520, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 520 (delta 3), reused 9 (delta 0), pack-reused 489
Receiving objects: 100% (520/520), 43.93 MiB | 61.29 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [5]:
!cd digitalclone-backend/ && git pull

Already up to date.


In [18]:
location_metaData = "/content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdata2/metaData_list_all.txt"
dataset_path_raw = "/content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdata2/raw"
dataset_path_clean = "/content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdata2/clean/wavs"
dataset_path = "/content/digitalclone-backend/aws_jobs_voiceclone/tts_tests/testdata2/clean/"
output_path = "/content/trainoutput"

In [19]:
# get list of files in audiofiles/wavs
audiofiles_raw = os.listdir(dataset_path_raw)
print('audiofiles...:', audiofiles_raw)


audiofiles...: ['list8-q3_1671048214796.wav', 'list4-q2_1671048494898.wav', 'list9-q1_1671048110495.wav', 'list4-q4_1671048504109.wav', 'list1-q8_1671048704560.wav', 'list3-q3_1671048556054.wav', 'list7-q9_1671048323379.wav', 'list5-q6_1671048454713.wav', 'list2-q3_1671048619283.wav', 'list6-q4_1671048372452.wav', 'list1-q9_1671048709584.wav', 'list8-q4_1671048220444.wav', 'list4-q3_1671048499833.wav', 'list9-q10_1671048175898.wav', 'list3-q2_1671048550054.wav', 'list5-q1_1671048425822.wav', 'list2-q8_1671048645021.wav', 'list6-q8_1671048402936.wav', 'list2-q4_1671048624226.wav', 'list5-q3_1671048436469.wav', 'list7-q5_1671048299610.wav', 'list6-q6_1671048382963.wav', 'list3-q9_1671048592548.wav', 'list2-q7_1671048639737.wav', 'list1-q3_1671048679438.wav', 'list9-q3_1671048131818.wav', 'list1-q4_1671048684115.wav', 'list7-q8_1671048317827.wav', 'list7-q2_1671048283567.wav', 'list1-q5_1671048689804.wav', 'list1-q7_1671048699715.wav', 'list3-q1_1671048544906.wav', 'list9-q8_1671048159979

In [8]:
# loop through each file in audiofiles/wavs and convert to 22050
for file in audiofiles_raw:
    # get file name
    filename = file.split(".")[0]
    # get file extension
    fileext = file.split(".")[1]
    # create command to convert file to 22050
    command = "ffmpeg -i " + dataset_path_raw + "/" + file + " -ar 16000 -ac 1 " + dataset_path_clean + "/" + filename + '.' + fileext + " -y"
    # run command
    os.system(command)
    # print
    print("converted " + file + " to 16000")

converted list8-q3_1671048214796.wav to 16000
converted list4-q2_1671048494898.wav to 16000
converted list9-q1_1671048110495.wav to 16000
converted list4-q4_1671048504109.wav to 16000
converted list1-q8_1671048704560.wav to 16000
converted list3-q3_1671048556054.wav to 16000
converted list7-q9_1671048323379.wav to 16000
converted list5-q6_1671048454713.wav to 16000
converted list2-q3_1671048619283.wav to 16000
converted list6-q4_1671048372452.wav to 16000
converted list1-q9_1671048709584.wav to 16000
converted list8-q4_1671048220444.wav to 16000
converted list4-q3_1671048499833.wav to 16000
converted list9-q10_1671048175898.wav to 16000
converted list3-q2_1671048550054.wav to 16000
converted list5-q1_1671048425822.wav to 16000
converted list2-q8_1671048645021.wav to 16000
converted list6-q8_1671048402936.wav to 16000
converted list2-q4_1671048624226.wav to 16000
converted list5-q3_1671048436469.wav to 16000
converted list7-q5_1671048299610.wav to 16000
converted list6-q6_1671048382963.

In [20]:
tpower = 1.3
tpreemphasis = 0.98
tdb = 20
######################


dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train=location_metaData, path=os.path.join(output_path, dataset_path)
)

audio_config = BaseAudioConfig(
    sample_rate=16000, 
    win_length=1024, 
    hop_length=256, 
    num_mels=80, 
    mel_fmin=0, 
    mel_fmax=None, 
    power=tpower,
    preemphasis=tpreemphasis,
    ref_level_db=tdb
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech",
    batch_size=32,
    eval_batch_size=16,
    batch_group_size=5,
    num_loader_workers=8,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False
)

In [ ]:
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    # eval_split_size=config.eval_split_size
    eval_split_size=0.1
)

ap = AudioProcessor.init_from_config(config)
tokenizer, config = TTSTokenizer.init_from_config(config)
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [ ]:
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

trainer.fit()

In [ ]:
## test it!

In [ ]:
!tts --text "this is my new voice, lets see how it works. Maybe it does not sound like me." \
      --model_path '/content/trainoutput/vits_ljspeech-December-17-2022_12+05AM-0000000/best_model_300.pth' \
      --config_path '/content/trainoutput/vits_ljspeech-December-17-2022_12+05AM-0000000/config.json' \
      --out_path 'out.wav'